<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

Homework8 Школа DataScientist, декабрь 2022, Андреева Ольга

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore', category=Warning)

In [2]:
c_RANDOM_STATE = 42

results_regression = pd.DataFrame(columns = ['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns = ['model', 'task', 'f1', 'accuracy'])

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('./csv4ml/boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [4]:
from sklearn.model_selection import train_test_split

y = data['MEDV'] #целевая переменная
X = data.drop(columns=['MEDV']) #все остальное

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=c_RANDOM_STATE)

print (len(X_train)/len(data), len(X_test)/len(data))

0.7984189723320159 0.2015810276679842


2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score

#Cтандартная регрессия
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)
y_pred = model_lr.predict(X_test)

r2_lr = r2_score(y_test, y_pred)

#Ridge
model_ridge = Ridge()
model_ridge.fit(X_train, y_train)
y_pred = model_ridge.predict(X_test)

r2_ridge =  r2_score(y_test, y_pred)

#Lasso
model_lasso = Lasso()
model_lasso.fit(X_train, y_train)
y_pred = model_lasso.predict(X_test)

r2_lasso =  r2_score(y_test, y_pred)


results_regression.loc[0] = ['LR', 'task2', r2_lr]
results_regression.loc[1] = ['Ridge', 'task2', r2_ridge]
results_regression.loc[2] = ['Lasso', 'task2', r2_lasso]

results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869


3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV

# значения для подбора коэффициента регуляризации
p_alphas=[10**x for x in range(-5,5)]


#1) GridSearchCV
#Для Ridge
params = {'alpha': p_alphas} 
model_ridge_grid = GridSearchCV(Ridge(), params)
model_ridge_grid.fit(X_train, y_train)
y_pred = model_ridge_grid.predict(X_test)
r2_ridge_grid_search =  r2_score(y_test, y_pred)

#Для Lasso
params = {'alpha': p_alphas}
model_lasso_grid = GridSearchCV(Lasso(), params)
model_lasso_grid.fit(X_train, y_train)
y_pred = model_lasso_grid.predict(X_test)
r2_lasso_grid_search =  r2_score(y_test, y_pred)


#2) RidgeCV 
model_ridge_cv = RidgeCV(alphas=p_alphas)
model_ridge_cv.fit(X_train, y_train)
y_pred = model_ridge_cv.predict(X_test)
r2_ridge_cv =  r2_score(y_test, y_pred)

#3) LassoCV
model_lasso_cv = LassoCV(alphas=p_alphas)
model_lasso_cv.fit(X_train, y_train)
y_pred = model_lasso_cv.predict(X_test)
r2_lasso_cv =  r2_score(y_test, y_pred)


results_regression.loc[3] = ['Ridge_GridSearchCV', 'task3', r2_ridge_grid_search]
results_regression.loc[4] = ['RidgeCV', 'task3', r2_ridge_cv]
results_regression.loc[5] = ['Lasso_GridSearchCV', 'task3', r2_lasso_grid_search]
results_regression.loc[6] = ['LassoCV', 'task3', r2_lasso_cv]

results_regression

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668475
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483


4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

pipelines= []
pipelines.append(Pipeline([('scaler', StandardScaler()),('moldel', Ridge())]))
pipelines.append(Pipeline([('scaler', MinMaxScaler()),('moldel', Ridge())]))
pipelines.append(Pipeline([('scaler', StandardScaler()),('moldel', Lasso())]))
pipelines.append(Pipeline([('scaler', MinMaxScaler()),('moldel', Lasso())]))

r2 = []

for pipe in pipelines:
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    r2.append(r2_score(y_test, y_pred))


results_regression.loc[7] = ['Ridge_StandardScaler', 'task4', r2[0]]
results_regression.loc[8] = ['Ridge_MinMaxScaler', 'task4', r2[1]]
results_regression.loc[9] = ['Lasso_StandardScaler', 'task4', r2[2]]
results_regression.loc[10] = ['Lasso_MinMaxScaler', 'task4', r2[3]]


results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668475
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483
7,Ridge_StandardScaler,task4,0.668190
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045


5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [8]:
# значения для подбора коэффициента регуляризации
p_alphas=[10**x for x in range(-5,5)]

pipelines= []
pipelines.append(Pipeline([('scaler', StandardScaler()),('moldel', RidgeCV(alphas=p_alphas))]))
pipelines.append(Pipeline([('scaler', MinMaxScaler()),('moldel',RidgeCV(alphas=p_alphas))]))
pipelines.append(Pipeline([('scaler', StandardScaler()),('moldel', LassoCV(alphas=p_alphas))]))
pipelines.append(Pipeline([('scaler', MinMaxScaler()),('moldel', LassoCV(alphas=p_alphas))]))

r2 = []

for pipe in pipelines:
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    r2.append(r2_score(y_test, y_pred))


results_regression.loc[11] = ['Ridge_StandardScaler_CV', 'task5', r2[0]]
results_regression.loc[12] = ['Ridge_MinMaxScaler_CV', 'task5', r2[1]]
results_regression.loc[13] = ['Lasso_StandardScaler_CV', 'task5', r2[2]]
results_regression.loc[14] = ['Lasso_MinMaxScaler_CV', 'task5', r2[3]]

results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668475
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483
7,Ridge_StandardScaler,task4,0.668190
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045


6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [9]:
from sklearn.preprocessing import PolynomialFeatures

pnf=PolynomialFeatures(2)

pipelines= []
pipelines.append(Pipeline([("pn_features", pnf), ('scaler', StandardScaler()),('moldel', Ridge())]))
pipelines.append(Pipeline([("pn_features", pnf), ('scaler', MinMaxScaler()),('moldel', Ridge())]))
pipelines.append(Pipeline([("pn_features", pnf), ('scaler', StandardScaler()),('moldel', Lasso())]))
pipelines.append(Pipeline([("pn_features", pnf), ('scaler', MinMaxScaler()),('moldel', Lasso())]))

r2 = []

for pipe in pipelines:
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    r2.append(r2_score(y_test, y_pred))


results_regression.loc[15] = ['Ridge_StandardScaler_Poly', 'task6', r2[0]]
results_regression.loc[16] = ['Ridge_MinMaxScaler_Poly', 'task6', r2[1]]
results_regression.loc[17] = ['Lasso_StandardScaler_Poly', 'task6', r2[2]]
results_regression.loc[18] = ['Lasso_MinMaxScaler_Poly', 'task6', r2[3]]

results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668475
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483
7,Ridge_StandardScaler,task4,0.668190
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045


7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [10]:
p_alphas=[10**x for x in range(-5,5)]
pnf=PolynomialFeatures(2)


pipelines= []
pipelines.append(Pipeline([("pn_features", pnf), ('scaler', StandardScaler()), ('moldel', RidgeCV(alphas=p_alphas))]))
pipelines.append(Pipeline([("pn_features", pnf), ('scaler', MinMaxScaler()), ('moldel', RidgeCV(alphas=p_alphas))]))
pipelines.append(Pipeline([("pn_features", pnf), ('scaler', StandardScaler()), ('moldel', LassoCV(alphas=p_alphas))]))
pipelines.append(Pipeline([("pn_features", pnf), ('scaler', MinMaxScaler()), ('moldel', LassoCV(alphas=p_alphas))]))

r2 = []

for pipe in pipelines:
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    r2.append(r2_score(y_test, y_pred))

results_regression.loc[19] = ['Ridge_StandardScaler_Poly_CV', 'task7', r2[0]]
results_regression.loc[20] = ['Ridge_MinMaxScaler_Poly_CV', 'task7', r2[1]]
results_regression.loc[21] = ['Lasso_StandardScaler_Poly_CV', 'task7', r2[2]]
results_regression.loc[22] = ['Lasso_MinMaxScaler_Poly_CV', 'task7', r2[3]]

results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668475
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483
7,Ridge_StandardScaler,task4,0.668190
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045


8. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [11]:
from itertools import product

df_results = pd.DataFrame(columns = ['model', 'R2'])
p_alphas=[10**x for x in range(-5,5)]

models=[ Ridge(), RidgeCV(alphas=p_alphas), Lasso(),LassoCV(alphas=p_alphas)]
scalers=[None, StandardScaler(), MinMaxScaler()]
pnfs=[None, PolynomialFeatures(2)]

for model, scaler, pnf in (product(models, scalers, pnfs)):
    params = {'model':model, 'scaler':scaler,'PolynomialFeatures':pnf}
    pipe = Pipeline([("pn_features", pnf), ('scaler', scaler), ('moldel', model)])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    df_results = df_results.append({'model':params,'R2':r2}, ignore_index=True)

#pd.options.display.float_format ='{:,.20f}'.format
#df_results.sort_values(by='R2', ascending=False)

r2_best_model = df_results.R2.max()
best_params = df_results.loc[df_results.R2==r2_best_model]['model'].values
print('Параметры лучшей модели:\n', best_params)
results_regression.loc[23] = ['Best_Model', 'task8', r2_best_model]


Параметры лучшей модели:
 [{'model': Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001), 'scaler': StandardScaler(copy=True, with_mean=True, with_std=True), 'PolynomialFeatures': PolynomialFeatures(degree=2, include_bias=True, interaction_only=False,
                   order='C')}]


In [12]:
results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668475
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483
7,Ridge_StandardScaler,task4,0.668190
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045


http://archive.ics.uci.edu/ml/datasets/Adult

In [13]:
data = pd.read_csv('./csv4ml/adult.csv')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [15]:
y = data['class'] #целевая переменная
X = data.drop(columns=['class']) #все остальное

y = y.apply(lambda x: 1 if x=='<=50K' else 0)
#y.head(10)


10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [16]:
from sklearn. metrics import f1_score
from sklearn. metrics import accuracy_score

y_most_freq = [y.mode()[0]]*len(y)
f1_most_frequent = f1_score(y_most_freq, y)
acc_most_frequent = accuracy_score(y_most_freq, y)
results_classification.loc[0] = ['Most Frequent class', 'task10', f1_most_frequent, acc_most_frequent]
results_classification

,model,task,f1,accuracy
0,Most Frequent class,task10,0.8641,0.760718


11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [17]:
data.isnull().sum()
#везде 0, значит пропусков нет

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
class             0
dtype: int64

12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [18]:
clmns_num = X.select_dtypes(include='number').columns
clmns_ctgr = X.select_dtypes(include='object').columns
print(clmns_num)
print(clmns_ctgr)

Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')
Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object')


13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [33]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate

models=[LogisticRegression(), KNeighborsClassifier(), LinearSVC()]

num_transformer = Pipeline([('scaler', MinMaxScaler())])
ctgr_transformer = Pipeline([('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(
    transformers=[('num', num_transformer, clmns_num),('ctgr', ctgr_transformer, clmns_ctgr)])
 
f1=[]
acc=[]

for model in models:
    pipe = Pipeline([('preprocessor', preprocessor), ('classifier', model)])
    cv_res = cross_validate(pipe, X, y, scoring={'f1':'f1','acc':'accuracy'})
    f1.append(cv_res['test_f1'].mean())
    acc.append(cv_res['test_acc'].mean())


results_classification.loc[1] = ['LogisticRegression', 'task13', f1[0], acc[0]]
results_classification.loc[2] = ['KNeighborsClassifier', 'task13',  f1[1], acc[1]]
results_classification.loc[3] = ['LinearSVC', 'task13',  f1[2], acc[2]]

results_classification

,model,task,f1,accuracy
0,Most Frequent class,task10,0.864100,0.760718
1,LogisticRegression,task13,0.904817,0.850887
2,KNeighborsClassifier,task13,0.887125,0.824659
3,LinearSVC,task13,0.906263,0.852873
4,LogisticRegression_impute,task14,0.904275,0.849822
5,KNeighborsClassifier_impute,task14,0.887272,0.824884
6,LinearSVC_impute,task14,0.905499,0.851378
7,LogisticRegression_delete_missings,task15,0.904814,0.850887
8,KNeighborsClassifier_delete_missings,task15,0.887125,0.824659
9,LinearSVC_delete_missings,task15,0.906263,0.852873


14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [20]:
%%time
from sklearn.impute import SimpleImputer


models=[LogisticRegression(), KNeighborsClassifier(), LinearSVC()]

num_transformer = Pipeline([('scaler', MinMaxScaler())])
ctgr_transformer = Pipeline([('imputer', SimpleImputer(missing_values='?', strategy='most_frequent'))
                            ,('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(
    transformers=[('num', num_transformer, clmns_num),('ctgr', ctgr_transformer, clmns_ctgr)])

f1=[]
acc=[]

for model in models:
    pipe = Pipeline([('preprocessor', preprocessor), ('classifier', model)])
    cv_res = cross_validate(pipe, X, y, scoring={'f1':'f1','acc':'accuracy'})
    f1.append(cv_res['test_f1'].mean())
    acc.append(cv_res['test_acc'].mean())

                            
results_classification.loc[4] = ['LogisticRegression_impute', 'task14', f1[0], acc[0]]
results_classification.loc[5] = ['KNeighborsClassifier_impute', 'task14', f1[1], acc[1]]
results_classification.loc[6] = ['LinearSVC_impute', 'task14', f1[2], acc[2]]
                            
results_classification                            

Wall time: 3min 13s


,model,task,f1,accuracy
0,Most Frequent class,task10,0.864100,0.760718
1,LogisticRegression,task13,0.904817,0.850887
2,KNeighborsClassifier,task13,0.887125,0.824659
3,LinearSVC,task13,0.906263,0.852873
4,LogisticRegression_impute,task14,0.904275,0.849822
5,KNeighborsClassifier_impute,task14,0.887272,0.824884
6,LinearSVC_impute,task14,0.905499,0.851378


15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [21]:
%%time
from sklearn.preprocessing import FunctionTransformer

models=[LogisticRegression(), KNeighborsClassifier(), LinearSVC()]

FunctionTransformer(lambda x: x[numeric], validate=False)
num_transformer = Pipeline([('scaler', MinMaxScaler())])
ctgr_transformer = Pipeline([('imputer', SimpleImputer(missing_values='?', strategy='constant', fill_value=None))
                            #,('drop', FunctionTransformer(lambda x: x.dropna())) 
                            ,('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(
    transformers=[('num', num_transformer, clmns_num),('ctgr', ctgr_transformer, clmns_ctgr)])

f1=[]
acc=[]

for model in models:
    pipe = Pipeline([('preprocessor', preprocessor), ('classifier', model)])
    cv_res = cross_validate(pipe, X, y, scoring={'f1':'f1','acc':'accuracy'})
    f1.append(cv_res['test_f1'].mean())
    acc.append(cv_res['test_acc'].mean())
    print(f1[-1], acc[-1])

results_classification.loc[7] = ['LogisticRegression_delete_missings', 'task15',  f1[0], acc[0]]
results_classification.loc[8] = ['KNeighborsClassifier_delete_missings', 'task15',  f1[1], acc[1]]
results_classification.loc[9] = ['LinearSVC_delete_missings', 'task15',  f1[2], acc[2]]
results_classification

0.9048139733918862 0.8508865189628042
0.8871248478873279 0.8246590741201024
0.9062634279884371 0.8528725280383656
Wall time: 2min 53s


,model,task,f1,accuracy
0,Most Frequent class,task10,0.864100,0.760718
1,LogisticRegression,task13,0.904817,0.850887
2,KNeighborsClassifier,task13,0.887125,0.824659
3,LinearSVC,task13,0.906263,0.852873
4,LogisticRegression_impute,task14,0.904275,0.849822
5,KNeighborsClassifier_impute,task14,0.887272,0.824884
6,LinearSVC_impute,task14,0.905499,0.851378
7,LogisticRegression_delete_missings,task15,0.904814,0.850887
8,KNeighborsClassifier_delete_missings,task15,0.887125,0.824659
9,LinearSVC_delete_missings,task15,0.906263,0.852873


 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [25]:
%%time
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

models=[RandomForestClassifier(), GradientBoostingClassifier()]

num_transformer = Pipeline([('scaler', MinMaxScaler())])
ctgr_transformer = Pipeline([('imputer', SimpleImputer(missing_values='?', strategy='most_frequent'))
                            ,('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(
    transformers=[('num', num_transformer, clmns_num),('ctgr', ctgr_transformer, clmns_ctgr)])

f1=[]
acc=[]

for model in models:
    pipe = Pipeline([('preprocessor', preprocessor), ('classifier', model)])
    cv_res = cross_validate(pipe, X, y, scoring={'f1':'f1','acc':'accuracy'})
    f1.append(cv_res['test_f1'].mean())
    acc.append(cv_res['test_acc'].mean())

results_classification.loc[10] = ['RandomForestClassifier', 'task16', f1[0], acc[0]]
results_classification.loc[11] = ['GradientBoostingClassifier', 'task16', f1[1], acc[1]]
results_classification

Wall time: 41.7 s


,model,task,f1,accuracy
0,Most Frequent class,task10,0.864100,0.760718
1,LogisticRegression,task13,0.904817,0.850887
2,KNeighborsClassifier,task13,0.887125,0.824659
3,LinearSVC,task13,0.906263,0.852873
4,LogisticRegression_impute,task14,0.904275,0.849822
5,KNeighborsClassifier_impute,task14,0.887272,0.824884
6,LinearSVC_impute,task14,0.905499,0.851378
7,LogisticRegression_delete_missings,task15,0.904814,0.850887
8,KNeighborsClassifier_delete_missings,task15,0.887125,0.824659
9,LinearSVC_delete_missings,task15,0.906263,0.852873


17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [44]:
%%time
df_results = pd.DataFrame(columns = ['model', 'perprocessor', 'f1', 'acc'])

models=[#LogisticRegression()
       # , KNeighborsClassifier()
       # , LinearSVC()
      #  , RandomForestClassifier()
        GradientBoostingClassifier()
       ]
scalers=[None
         , StandardScaler()
         , MinMaxScaler()]
imputers = [None
            , SimpleImputer(missing_values='?', strategy='most_frequent')
            , SimpleImputer(missing_values='?', strategy='constant', fill_value=None)
           ]
onehots = [OneHotEncoder(handle_unknown='ignore')]

i=0
f1=[]
acc=[]
for model, scaler, imputer, onehot in (product(models, scalers, imputers, onehots)):
    
    num_params = [('scaler', scaler)]
    ctgr_params = [('imputer', imputer), ('onehot', onehot)]
    
    num_transformer = Pipeline(num_params)
    ctgr_transformer = Pipeline(ctgr_params)
    
    preprocessor = ColumnTransformer(
        transformers=[('num', num_transformer, clmns_num),('ctgr', ctgr_transformer, clmns_ctgr)])


    pipe = Pipeline([('preprocessor', preprocessor), ('classifier', model)])
    cv_res = cross_validate(pipe, X, y, scoring={'f1':'f1','acc':'accuracy'})
    f1 =cv_res['test_f1'].mean()
    acc = cv_res['test_acc'].mean()
    i+=1
    print   (i, f1[-1], acc[-1])
    
    df_results = df_results.append({'model':model, 'perprocessor':preprocessor, 'f1':f1, 'acc':acc}, ignore_index=True)
df_results
#best_params = {}
print('Параметры лучшей модели:\n', best_params)
#f1_best = 0
#acc_best = 0
#results_classification.loc[12] = ['Best_Model', 'task17', f1_best, acc_best]

IndexError: invalid index to scalar variable.

In [ ]:
results_classification